In [18]:
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [19]:
import numpy as np
from keras import layers
from keras.initializers import glorot_uniform
from keras.models import Model, load_model
import tensorflow as tf

In [20]:
def identityblock(X,f,filters,stage,block):
    X_shortcut=X
    conv_name='res'+str(stage)+block+'_branch'
    batch_name='bn'+str(stage)+block+'_branch'
    F1,F2,F3=filters
    ##Stage 1
    X=Conv2D(filters=F1,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name+'2a',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=batch_name+'2a')(X)
    X=Activation('relu')(X)
    
    ##Stage 2
    X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',name=conv_name+'2b',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=batch_name+'2b')(X)
    X=Activation('relu')(X)
    
    ##Stage 3
    X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name+'2c',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=batch_name+'2c')(X)
    
    X=layers.Add()([X,X_shortcut])
    X=Activation('relu')(X)
    
    return X

In [21]:
def convolutionblock(X,f,filters,stage,block,s=2):
    F1,F2,F3=filters
    X_shortcut=X
    conv_name='res'+str(stage)+block+'_branch'
    batch_name='bn'+str(stage)+block+'_branch'
    
    ##Main path
    X=Conv2D(filters=F1,kernel_size=(1,1),strides=(s,s),name=conv_name+'2a',kernel_initializer=glorot_uniform(seed=0))(X)  
    X=BatchNormalization(axis=3,name=batch_name+'2a')(X)
    X=Activation('relu')(X)
    
    ##2md stage of main path
    X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',name=conv_name+'2b',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=batch_name+'2b')(X)
    X=Activation('relu')(X)
    
    ###3rd stage of main path
    X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),name=conv_name+'2c',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=batch_name+'2c')(X)
    
    ##Shotcut making
    X_shortcut=Conv2D(F3,(1,1),strides=(s,s),name=conv_name+'1',kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut=BatchNormalization(axis=3,name=batch_name+'1')(X_shortcut)
    
    X=layers.Add()([X,X_shortcut])
    X=Activation('relu')(X)
    
    return X
    


In [26]:
def ress50(input_shape=(64,64,3),classes=6):
    X_input=Input(input_shape)
    X=ZeroPadding2D((3,3))(X_input)
    ##Stage 1
    X=Conv2D(64,(7,7),strides=(2,2),name='conv1',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name='bn_conv1')(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((3,3),strides=(2,2))(X)
    
    ##def identityblock(X,f,filters,stage,block):
    ##def convolutionblock(X,f,filters,stage,block,s=2):
    ##Stage 2
    X=convolutionblock(X,f=3,filters=[64,64,256],stage=2,block='a',s=1)
    X=identityblock(X,3,[64,64,256],stage=2,block='b')
    X=identityblock(X,3,[64,64,256],stage=2,block='c')
    
    ##stage 3
    X=convolutionblock(X,f=3,filters=[128,128,512],stage=3,block='a',s=2)
    X=identityblock(X,3,[128,128,512],stage=3,block='b')
    X=identityblock(X,3,[128,128,512],stage=3,block='c')
    X=identityblock(X,3,[128,128,512],stage=3,block='d')
    
    ##Stage 4
    X=convolutionblock(X,f=3,filters=[256,256,1024],stage=4,block='a',s=2)
    X=identityblock(X,3,[256,256,1024],stage=4,block='b')
    X=identityblock(X,3,[256,256,1024],stage=4,block='c')
    X=identityblock(X,3,[256,256,1024],stage=4,block='d')
    X=identityblock(X,3,[256,256,1024],stage=4,block='e')
    X=identityblock(X,3,[256,256,1024],stage=4,block='f')
    
    ##Stage 5
    X=convolutionblock(X,f=3,filters=[512,512,2048],stage=5,block='a',s=2)
    X=identityblock(X,3,[512,512,2048],stage=5,block='b')
    X=identityblock(X,3,[512,512,2048],stage=5,block='c')
    
    
    X=AveragePooling2D((2,2),name='avg_pool')(X)
    X=Flatten()(X)
    
    X=Dse(classes,activation='softmax',name='fc'+str(classes),kernel_initializer=glorot_uniform(seed=0))(X)
    
    
    
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    return model
    

In [27]:
model=ress50(input_shape=(64,64,3),classes=6)

In [28]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [30]:
import h5py

In [31]:
def load_data():
    data=h5py.File('D:\\programs\\Excel files\\signs dataset\\train_signs.h5',"r")
    X_train_or=np.array(data["train_set_x"][:])
    Y_train_or=np.array(data["train_set_y"][:])
    
    data1=h5py.File('D:\\programs\\Excel files\\signs dataset\\test_signs.h5',"r")
    X_test_or=np.array(data1["test_set_x"][:])
    Y_test_or=np.array(data1["test_set_y"][:])
    
    return X_train_or,Y_train_or,X_test_or,Y_test_or

In [32]:
X_train1,Y_train1,X_test1,Y_test1=load_data()

In [34]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [35]:
X_train = X_train1/255.
X_test = X_test1/255.
Y_train = convert_to_one_hot(Y_train1, 6).T
Y_test = convert_to_one_hot(Y_test1, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [36]:
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

Epoch 1/2
34/34 [==============================] - 77s 2s/step - loss: 1.7631 - accuracy: 0.4870
Epoch 2/2
34/34 [==============================] - 73s 2s/step - loss: 0.9738 - accuracy: 0.7056


In [38]:
 model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 70, 70, 3)    0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________